# SNEWS Firedrill Test ~May 25th~ Feb 2023


## Installing Stuff

- [**SNEWS_PT Repo**](https://github.com/SNEWS2/SNEWS_Publishing_Tools) <br>
- [**hop repo**](https://github.com/scimma/hop-client) <br>
- [**hop auth**](https://my.hop.scimma.org/hopauth/)


Start by making sure your hop credentials are updated for firedrills:

snews.alert-firedrill
snews.experiments-firedrill
snews.mma-firedrill

- Make sure you are using hop 0.5.0

```pip install hop-client=0.5.0``` <br>
You can check your version with `hop --version`

Install SNEWS_PT

```pip install -U snews-pt```





# What is in this firedrill tutorial

This notebook has the following;<br>
- Subscribe methods for alerts, interactive notebooks and command line
- Publish observation messages, interactive notebooks and command line, different features
- Publish Heartbeat messages
- Test connection and Request feedback
- FIRE DRILL


## Using the API

In [ ]:
from snews_pt.snews_pt_utils import set_name                       # to set experiment name
from snews_pt.snews_pub import SNEWSTiersPublisher                 # to publish observations & heartbeats
from snews_pt.snews_sub import Subscriber                          # to listen alerts
from snews_pt.remote_commands import test_connection, get_feedback # to test connection and get feedback
from datetime import  datetime
from time import sleep

The experiment can set their detector name. Once this is set, the software no longer raises warnings. 

In [ ]:
set_name()

You can subscribe for the alerts in a jupyter notebook, however, since this opens a stream and continuosly listen, you can not use the other cells without killing it. Therefore, we recommend to subscribe using a bash terminal via the following command
```bash
user:~$ snews_pt subscribe
```

In [ ]:
# this runs persistently
Subscriber(firedrill_mode=True).subscribe(outputfolder='./')

## Create and Publish Observation messages

In [ ]:
observation_time = "2023-06-14T12:45:45.100000"
msg = SNEWSTiersPublisher(neutrino_time=observation_time, is_test=True, firedrill_mode=True)

#### SNEWSTierPublisher
The `SNEWSTierPublisher` creates a message for you from your given keys and decides what to do with this message.

The `neutrino_time` is the most important argument. The Publisher decides that this message is intended for the `CoincidenceTier` based on this input. <br>

Try calling the function with no argument -> `SNEWSTiersPublisher()` <br>
This returns `"No valid message is created! Check your input and try again!"` <br>

Another example would be the heartbeat messages <br>
If you call it with a detector status i.e. `SNEWSTiersPublisher(detector_status='ON')` this should create a message for the heartbeats. What happens if you pass both neutrino times and detector status? Try it out!

----

Few more tips;<br>
if you pass `neutrino_time` the Publisher labels it as **"Coincidence Tier"** <br>
if you pass `p_values` and `t_bin_width` it is labeled as **"Significance Tier Message"** <br>
if you pass `timing_series` it is labeled as **"Time Tier"**<br>
if you pass `retract_latest` (e.g. retract_latest=1 retracts the last 1 message) it labels as **"Retraction Message"**<br>
if you pass `detector_status` it labels as **"Heartbeat Message"** (Notice HB doesn't require time as it stamps itself)

---

The `is_test` argument is needed so that coincidence algorithm knows we are testing and ignores even if the times that are passed illogical (e.g. from past or from far future). If this is not passed, Publisher will tell you that this message is not valid.

In [ ]:
## Here try and play with different arguments
# SNEWSTiersPublisher()

Notice that you only **create** the message and not send it anywhere yet! <br>
Once created, you can still display and modify the messages. See the example below.

In [ ]:
msg = SNEWSTiersPublisher(neutrino_time=observation_time, is_test=True, firedrill_mode=True, detector_status='ON')

In [ ]:
# see the selected tiers
print(msg.tiernames)
# see the message data
msg.message_data

In [ ]:
# the first message that is created
msg.messages[0]

In [ ]:
# the second message that is created
msg.messages[1]

One can see that there are some more information added automatically. Now let's **send these messages to snews**

In [ ]:
msg.send_to_snews()

**Another trick** <br>
If you think that you have some other information that you think is relevant, you can still pass any other meta data by passing them as key-word arguments.

In [ ]:
msg2 = SNEWSTiersPublisher(neutrino_time=observation_time, is_test=True, comment='We were taking calibration data')
msg2.messages[0]

### JSON messages and Observation Publication via CLI

Similarly, these observation messages can also be sent with the Command Line Interface (CLI). <br>
This requires file(s) that are properly constructed and formatted as a json file then these file(s) can be passed from the terminal <br>

For this assume we have the `"observation_file.json"` which contains
```JSON
{
    "machine_time": 
        "2023-02-06T10:18:44.948273",
    "neutrino_time": 
        "2023-02-06T10:18:44.948273",
    "p_val":
        0.07,
    "something extra":
        "Our neutrino time is defined as the first event above bg"
}
```

```bash 
user:~$ snews_pt observation_file.json
```

In [ ]:
# json_message = SNEWSTiersPublisher.from_json("observation_file.json")
# json_message.send_to_snews()

## Heartbeats

SNEWS tracks several time informations with different meanings and purposes; <br>
- `neutrino_time` : this is used for searching coincidences and refers to exact time of the first neutrino detection <br>
- `machine_time` : this is an optional argument to indicate the time labeled by the machine, ideally for the observation message this would be the same as the neutrino time. Howeever, for the heartbeat messages it should be the time that detector heartbeat was checked. 
- `sent_time` : this is added by the `SNEWSTierPublisher().send_to_snews()` upon execution. We use this information for latency measurements. <br>

On the server side, we also label messages by their `received_time` which allows us (for example for the heartbeats) to calculate the latency between the time detector checks their status, the time they actually send it, and the time we receive it at the server.  

In [ ]:
# simply create a message for the heartbeat
msg_hb = SNEWSTiersPublisher(detector_status="ON")

Initially, the `sent_time` is set to current time i.e. same as `machine_time` however, when the message is sent, it is overwritten with the exact time at the time of execution.

In [ ]:
msg_hb.messages[0]

In [ ]:
msg_hb.send_to_snews()

### Heartbeat CLI 

```bash 
user:~$ snews_pt heartbeat --status ON --time "2023-02-06T10:40:59.675226"
```
where the `--time` refers to `machine_time` and is optional. (By default it is set equal to `sent_time` which is stamped upon execution).

## Other features

We try to provide some more tools that are convenient for the user such as testing the connection and requesting feedback on their last 24 hour heartbeat status. <br>
The `test_connection` function injects a message into the stream with your detector name and current time, also labels itself as `"status":"sending"`. If you have a connection to the server (and the server is actually running), it should be received, understood, and new message should be created with the same properties except this time `"status":"confirmed"` as its label. This time server injects this into the stream, and on the user's end this message is searched for. If the `test_connection` finds this message in the stream within pre-set time, it informs the user that the connection was established.

In [ ]:
test_connection()

In [ ]:
# now I start the server and test again
test_connection()

### Request Feedback

At the server we keep the heartbeats in several different files. At any given time, the records of the last 24 hours is kept. Every day, the heartbeats of a given day is also stored separately and deleted after 7 days. <br>
Authorized users can request feedback on their heartbeats within the last 24hours. This selects the data belonging to your experiment and computes some statistics regarding the status, arrival times and latencies and sends an email to your requested address(es) **only if** those emails are already registered by us under that experiment.   <br>
This way, we prevent non-member and non-authorized people from accessing the heartbeat information of another experiment.

In [ ]:
get_feedback(detector_name="XENONnT", email_address="kara@kit.edu")

## Bonus, integrating your custom scripts

If you have a follow-up script which uses the alert content and does more calculations, you can plug this script to subscribe function
```bash
snews_pt subscribe -p mycustomscript.py
```

> The `mycustomscript.py` should contain 
> ```python
> import sys, json
> data = json.load(open(sys.argv[1]))
> ``` 
> then the `data` contains the alert content as a dictionary object.

---
# Fire Drill

Now let's run a fire drill together. 
- 1. Test your connection
- 2. Open a separate notebook and start sending heartbeats

In [ ]:
from snews_pt.remote_commands import test_connection
from snews_pt.snews_pub import SNEWSTiersPublisher

In [ ]:
test_connection()

In [ ]:
import time
for i in range(7):
    time.sleep(2)
    msg = SNEWSTiersPublisher(detector_status='ON', firedrill_mode=True) 
    msg.send_to_snews()

- 3. Create an observation message for your detector based on the times given below

```json
{"JUNO": {"neutrino_time": "2023-02-07T12:00:00.000000"}}, 
{"HK": {"neutrino_time": "2023-02-07T12:00:00.001076"}}, 
{"KL": {"neutrino_time": "2023-02-07T12:00:00.001090"}}, 
{"SK": {"neutrino_time": "2023-02-07T12:00:00.001090"}}, 
{"PandaX-4T": {"neutrino_time": "2023-02-07T12:00:00.002390"}}, 
{"Baksan": {"neutrino_time": "2023-02-07T12:00:00.018629"}}, 
{"IC": {"neutrino_time": "2023-02-07T12:00:00.023424"}}, 
{"LVD": {"neutrino_time": "2023-02-07T12:00:00.026127"}}, 
{"DS-20K": {"neutrino_time": "2023-02-07T12:00:00.026127"},
{"Borexino":{"neutrino_time": "2023-02-07T12:00:00.026127"}}, 
{"XENONnT": {"neutrino_time": "2023-02-07T12:00:00.026127"}}
{"KM3": {"neutrino_time": "2023-02-07T12:00:00.026338"}},
{"NOvA": {"neutrino_time": "2023-02-07T12:00:00.028417"}}, 
{"HALO-1kT": {"neutrino_time": "2023-02-07T12:00:00.030543"}}, 
{"SNOP": {"neutrino_time": "2023-02-07T12:00:00.030543"}}, 
{"SBND": {"neutrino_time": "2023-02-07T12:00:00.030548"}}, 
{"DUNE": {"neutrino_time": "2023-02-07T12:00:00.030548"}}, 
{"LZ": {"neutrino_time": "2023-02-07T12:00:00.030548"}}, 
{"MicroBooNe": {"neutrino_time": "2023-02-07T12:00:00.030548"}}
```

These times should point to the candidate star Alf Ori at <br>RA: 88.79292 deg<br>
DEC: 7.40706 deg

In [ ]:
msg = SNEWSTiersPublisher(neutrino_time="", is_test=True, firedrill_mode=True)

- 4. On a separate notebook or a terminal subscribe to the alerts
```bash
 user:~$ snews_pt subscribe
```
or 

```python
from snews_pt.snews_sub import Subscriber
Subscriber().subscribe()
```

- 5. send your observation to snews

In [ ]:
msg.send_to_snews()